In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# !pip install ydata-profiling

In [ ]:
# from ydata_profiling import ProfileReport

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/ML_OBESITY_PROJECT/obesity_level.csv')
test_df=pd.read_csv('/content/drive/MyDrive/ML_OBESITY_PROJECT/test.csv')


In [ ]:
# profile=ProfileReport(train_df,title='Pandas Profiling Report',explorative=True)
# profile.to_notebook_iframe()

In [ ]:
train_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,0be1dad
0,0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,Sometimes,0,2.763573,0,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,Frequently,0,2.000000,0,1.000000,1.000000,no,Automobile,0rmal_Weight
2,2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,Sometimes,0,1.910378,0,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,Sometimes,0,1.674061,0,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,Sometimes,0,1.979848,0,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [ ]:
# test_df.head()

In [ ]:
train_df=train_df.drop("id",axis='columns')
test_df=test_df.drop("id",axis='columns')

In [ ]:
train_df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,0be1dad
0,Male,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,Sometimes,0,2.763573,0,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,Frequently,0,2.000000,0,1.000000,1.000000,no,Automobile,0rmal_Weight
2,Female,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,Sometimes,0,1.910378,0,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,Sometimes,0,1.674061,0,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,Sometimes,0,1.979848,0,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [ ]:
x=train_df.drop("0be1dad",axis='columns')
y=train_df['0be1dad']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
numeric_features = ['Age', 'Height','Weight','FCVC','NCP','CH2O','FAF','TUE']


In [ ]:
transformer = ColumnTransformer(transformers=[
    ('t1',OneHotEncoder(drop='first'),['CAEC']),
    ('t2',OneHotEncoder(drop='first'),['CALC']),
    ('t3', StandardScaler(), numeric_features),

    ('t4',OneHotEncoder(drop='first', handle_unknown='ignore'),['Gender','family_history_with_overweight','FAVC','SMOKE','SCC','MTRANS'])
],remainder='passthrough')

In [ ]:
y = np.array(y)
y.shape

(20758,)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

encoder = OrdinalEncoder()
y_encoded = encoder.fit_transform(np.array(y).reshape(-1, 1))
y_encoded.shape

(20758, 1)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 475, 485),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.065),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 5),
        'gamma': trial.suggest_float('gamma', 0.2, 0.4),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.4, 0.6),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 0.5),
    }

    # Create the XGBoost model
    model = XGBClassifier(**params, objective='multi:softmax') # Removed num_class

    # Evaluate the model using cross-validation
    score = cross_val_score(model, x_transformed, y_encoded, cv=3, scoring='accuracy').mean() # Use original y

    return score


best_params={'n_estimators': 476,  'learning_rate': 0.056175675016670416, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.3354617624351886, 'subsample': 0.7871797069127701, 'colsample_bytree': 0.5071468557602662, 'reg_alpha': 0.5393018307093757, 'reg_lambda': 0.0809287158560023}

Best hyperparameters: {'n_estimators': 473, 'learning_rate': 0.056175675016670416, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.3354617624351886, 'subsample': 0.7871797069127701, 'colsample_bytree': 0.5071468557602662, 'reg_alpha': 0.5393018307093757, 'reg_lambda': 0.0809287158560023}
Best accuracy: 0.91294

In [ ]:
x_transformed = transformer.fit_transform(x)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Create an Optuna study
# study = optuna.create_study(direction='maximize')

# # Encode the target variable y into integer labels
label_encoder = LabelEncoder()
y_encoded_labels = label_encoder.fit_transform(y)

# # Run the optimization with more trials
# study.optimize(lambda trial: objective(trial), n_trials=100) # Increased number of trials

# # Print the best hyperparameters and the best score

# print("Best hyperparameters:", study.best_params)
# print("Best accuracy:", study.best_value)
best_params={'n_estimators': 483,  'learning_rate': 0.056175675016670416, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.3354617624351886, 'subsample': 0.7871797069127701, 'colsample_bytree': 0.5071468557602662, 'reg_alpha': 0.5393018307093757, 'reg_lambda': 0.0809287158560023}
final_model = XGBClassifier(**best_params, random_state=42, objective='multi:softmax', num_class=len(label_encoder.classes_))
final_model.fit(x_transformed, y_encoded_labels)

# Make predictions on the training data to evaluate accuracy
y_pred = final_model.predict(x_transformed)

# Calculate and print the accuracy
accuracy = accuracy_score(y_encoded_labels, y_pred)
print(f"Accuracy on the training data: {accuracy}")


# Train the final model with the best hyperparameters
# final_model = XGBClassifier(**study.best_params, random_state=42, objective='multi:softmax', num_class=len(label_encoder.classes_))
# final_model.fit(x_transformed, y_encoded_labels)

Accuracy on the training data: 0.9355910974082281


In [ ]:
# Train the final model with the best hyperparameters


# Fit the transformer on the training data
x_transformed = transformer.fit_transform(x)

# Transform the test data
test_transformed = transformer.transform(test_df)

# Make predictions on the test data
test_predictions_encoded = final_model.predict(test_transformed)

# Decode the predictions
test_predictions = encoder.inverse_transform(test_predictions_encoded.reshape(-1, 1))

# Create the submission DataFrame
submission_df = pd.DataFrame({'id': test_df.index + 15533, 'Weight_Category': test_predictions.flatten()})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('t1.csv', index=False)

print("submission.csv created successfully with predictions from the optimized model!")

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


submission.csv created successfully with predictions from the optimized model!


In [ ]:
# Check for unknown categories in test_df for the categorical features in the transformer
categorical_features_t4 = ['Gender','family_history_with_overweight','FAVC','SMOKE','SCC','MTRANS']
for col in categorical_features_t4:
    train_categories = set(x[col].unique())
    test_categories = set(test_df[col].unique())
    unknown_categories = test_categories - train_categories
    if unknown_categories:
        print(f"Unknown categories in column '{col}': {unknown_categories}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initialize the OrdinalEncoder to decode the predictions
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
# Fit the encoder on the training labels to ensure it has the correct mapping
encoder.fit(y.reshape(-1, 1))